In [ ]:
!pip install langgraph

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Literal

class AgentState(TypedDict):
    candidate_name: str
    objective: str
    thought: str
    next_action: str
    research_notes: List[str]
    tools_used: List[str]
    iteration: int
    final_report: str

def search_linkedin(candidate_name: str) -> str:
    print(f"Tool: Searching LinkedIn for {candidate_name}...")
    return f"Found LinkedIn profile: 5 years experience as a Python developer."

def search_github(candidate_name: str) -> str:
    print(f"Tool: Searching GitHub for {candidate_name}...")
    return f"Found GitHub: 45 repositories, mostly Python, active contributor."

def search_company_records(candidate_name: str) -> str:
    print(f"Tool: Searching company records for {candidate_name}...")
    return f"Previous application: Applied 2 years ago for a junior role."

def agent_think(state: AgentState) -> dict:
    print(f"\nAgent Thinking (Iteration {state['iteration'] + 1})...")
    tools_used = state["tools_used"]

    if "linkedin" not in tools_used:
        thought = "I'll start with LinkedIn to get basic professional info."
        next_action = "search_linkedin"
    elif "github" not in tools_used:
        thought = "LinkedIn looks good. Now, I'll verify technical skills on GitHub."
        next_action = "search_github"
    elif "company_records" not in tools_used:
        thought = "They seem strong. Let me check if they've applied before."
        next_action = "search_company_records"
    else:
        thought = "I have enough information to compile my findings."
        next_action = "finish"

    return {"thought": thought, "next_action": next_action}

def agent_act(state: AgentState) -> dict:
    action = state["next_action"]
    candidate = state["candidate_name"]
    print(f"Agent Acting: {action}")

    if action == "search_linkedin":
        result = search_linkedin(candidate)
        return {"research_notes": state["research_notes"] + [result], "tools_used": state["tools_used"] + ["linkedin"]}
    elif action == "search_github":
        result = search_github(candidate)
        return {"research_notes": state["research_notes"] + [result], "tools_used": state["tools_used"] + ["github"]}
    elif action == "search_company_records":
        result = search_company_records(candidate)
        return {"research_notes": state["research_notes"] + [result], "tools_used": state["tools_used"] + ["company_records"]}
    return {}

def create_final_report(state: AgentState) -> dict:
    print("\nCreating final report...")
    report = f"CANDIDATE RESEARCH REPORT: {state['candidate_name']}\n\nFINDINGS:\n" + "\n".join(f"- {note}" for note in state['research_notes'])
    return {"final_report": report}

def agent_should_continue(state: AgentState) -> Literal["continue", "finish"]:
    if state["next_action"] == "finish" or state["iteration"] >= 4:
        return "finish"
    return "continue"

def update_iteration(state: AgentState) -> dict:
    return {"iteration": state["iteration"] + 1}

workflow = StateGraph(AgentState)
workflow.add_node("think", agent_think)
workflow.add_node("act", agent_act)
workflow.add_node("update_iter", update_iteration)
workflow.add_node("report", create_final_report)

workflow.set_entry_point("think")
workflow.add_edge("think", "act")
workflow.add_edge("act", "update_iter")
workflow.add_conditional_edges(
    "update_iter",
    agent_should_continue,
    {"continue": "think", "finish": "report"}
)
workflow.add_edge("report", END)
app = workflow.compile()

initial_state = {
    "candidate_name": "John Developer",
    "objective": "Research candidate's technical background.",
    "research_notes": [],
    "tools_used": [],
    "iteration": 0
}

result = app.invoke(initial_state)
print("\n" + "="*60 + "\n" + result["final_report"] + "\n" + "="*60)


Agent Thinking (Iteration 1)...
Agent Acting: search_linkedin
Tool: Searching LinkedIn for John Developer...

Agent Thinking (Iteration 2)...
Agent Acting: search_github
Tool: Searching GitHub for John Developer...

Agent Thinking (Iteration 3)...
Agent Acting: search_company_records
Tool: Searching company records for John Developer...

Agent Thinking (Iteration 4)...
Agent Acting: finish

Creating final report...

CANDIDATE RESEARCH REPORT: John Developer

FINDINGS:
- Found LinkedIn profile: 5 years experience as a Python developer.
- Found GitHub: 45 repositories, mostly Python, active contributor.
- Previous application: Applied 2 years ago for a junior role.
